<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Analyzing Chipotle Data

_Author: Joseph Nelson (DC)_

---

For Project 2, you will complete a series of exercises exploring [order data from Chipotle](https://github.com/TheUpshot/chipotle), compliments of _The New York Times'_ "The Upshot."

For these exercises, you will conduct basic exploratory data analysis (Pandas not required) to understand the essentials of Chipotle's order data: how many orders are being made, the average price per order, how many different ingredients are used, etc. These allow you to practice business analysis skills while also becoming comfortable with Python.

---

## Basic Level

### Part 1: Read in the file with `csv.reader()` and store it in an object called `file_nested_list`.

Hint: This is a TSV (tab-separated value) file, and `csv.reader()` needs to be told [how to handle it](https://docs.python.org/2/library/csv.html).

In [38]:
import csv
from collections import namedtuple   # Convenient to store the data rows

DATA_FILE = './data/chipotle.tsv'


In [39]:
with open(DATA_FILE) as tabfile:
    read_file = csv.reader(tabfile, delimiter="\t")
    file_nested_list = list(read_file)

### Part 2: Separate `file_nested_list` into the `header` and the `data`.


In [56]:
#Identified header and data of file_nested_list by converting to DataFrame and assigning row [0] as header
import pandas as pd
dl = pd.DataFrame(file_nested_list)
dl.columns = dl.iloc[0]
df = dl.drop(dl.index[0])
df.head()

,order_id,quantity,item_name,choice_description,item_price
1,1,1,Chips and Fresh Tomato Salsa,NULL,$2.39
2,1,1,Izze,[Clementine],$3.39
3,1,1,Nantucket Nectar,[Apple],$3.39
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL,$2.39
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


In [58]:
#data preparation for analysis - remove '$' from item_price, 
#convert order_id, quantity and item_price to numeric
df['item_price'].replace({'\$':''}, inplace = True, regex=True)
df["quantity"] = df.quantity.astype(int, inplace = True)
df["order_id"] = df.order_id.astype(int, inplace = True)
df['item_price'] = df.item_price.astype(float, inplace = True)
df.dtypes

TypeError: Cannot compare types 'ndarray(dtype=float64)' and 'str'

---

## Intermediate Level

### Part 3: Calculate the average price of an order.

Hint: Examine the data to see if the `quantity` column is relevant to this calculation.

Hint: Think carefully about the simplest way to do this!

In [42]:
#Check that item_price takes quantity into account
df_chips = df.loc[df['item_name'] == 'Side of Chips']
print(df_chips.head())
print(df_chips.loc[df_chips['quantity'] > 1])

0    order_id  quantity      item_name choice_description  item_price
7           3         1  Side of Chips               NULL        1.69
33         16         1  Side of Chips               NULL        1.69
71         31         1  Side of Chips               NULL        1.69
150        67         1  Side of Chips               NULL        1.69
276       120         1  Side of Chips               NULL        1.69
0     order_id  quantity      item_name choice_description  item_price
2945      1170         3  Side of Chips               NULL        5.07
3888      1559         8  Side of Chips               NULL       13.52


from obversations, quantity of item is already factored into item_price, so quantity will be irrelevant to the calculation


In [54]:
# ii. Group numerical columns by 'order_id', with corresponding 'item_price' column, 
#     containing the sum of all item_prices with same order_id value.
# iii. calculate mean 'item_price' of 'order_id's, rounded to 2 decimal places.
dp = df.groupby('order_id').sum()
d_avg = round(dp['item_price'].mean(),2)
print('Average price of an order is $',d_avg)

Average price of an order is $ 18.81


### Part 4: Create a list (or set) named `unique_sodas` containing all of unique sodas and soft drinks that Chipotle sells.

Note: Just look for `'Canned Soda'` and `'Canned Soft Drink'`, and ignore other drinks like `'Izze'`.

In [44]:
dz = df[(df.item_name == 'Canned Soft Drink')|(df.item_name == 'Canned Soda')]
unique_sodas = set(dz['choice_description'])
print(list(unique_sodas))

['[Sprite]', '[Diet Dr. Pepper]', '[Lemonade]', '[Nestea]', '[Mountain Dew]', '[Dr. Pepper]', '[Coke]', '[Coca Cola]', '[Diet Coke]']


In [52]:
def get_sodas(orders):
    dz = orders[(orders.item_name == 'Canned Soft Drink')|(orders.item_name == 'Canned Soda')]
    unique_sodas = set(dz['choice_description'])
    print(list(unique_sodas))
    
get_sodas(df)

[]


---

## Advanced Level


### Part 5: Calculate the average number of toppings per burrito.

Note: Let's ignore the `quantity` column to simplify this task.

Hint: Think carefully about the easiest way to count the number of toppings!


In [51]:
df_burrito = df[df['item_name'].str.contains("Burrito")]

topping_list = df_burrito.choice_description.tolist()
no_of_toppings = []
#split each value in topping_list by ',', with each segment becoming an element in another list. 
#The length of each new list is the number of toppings on each burrito
#As this is all we need, we then stored the length of each list in no_of_toppings
for i in topping_list:
    no_of_toppings.append(len(i.split(",")))
#no. of toppings are discreet, so we make it an integer before averaging all list elements
average_toppings = int(sum(no_of_toppings)/len(no_of_toppings))
print('Average number of toppings per burrito is', average_toppings)

ZeroDivisionError: division by zero

### Part 6: Create a dictionary. Let the keys represent chip orders and the values represent the total number of orders.

Expected output: `{'Chips and Roasted Chili-Corn Salsa': 18, ... }`

Note: Please take the `quantity` column into account!

Optional: Learn how to use `.defaultdict()` to simplify your code.

In [50]:
import numpy as np
df['item_name'].replace({'-':' '}, inplace=True, regex=True)
dchips = df[df['item_name'].str.contains("Chip")]
#Remove'-' in item_name to avoid duplicates when pivoted
pivot_chips = pd.pivot_table(dchips, values='quantity', index=['item_name'], aggfunc=np.sum)
#reset pivot table index to gain access to item_name column
reset_chips = pivot_chips.reset_index()
dict_chips = dict(zip(reset_chips.item_name,reset_chips.quantity))
dict_chips

AttributeError: 'DataFrame' object has no attribute 'quantity'

---

## Bonus: Craft a problem statement about this data that interests you, and then answer it!


### Problem: Find top selling item
### Problem - find number of orders for any item 

In [49]:
import numpy as np
def get_orders(food):
    food_low = food.lower()
    df['item_name'] = df['item_name'].str.lower()
    df['item_name'].replace({'-':' '}, inplace=True, regex=True)
    d_order = df[df['item_name'].str.contains(food_low)]
    #Remove'-' in item_name to avoid duplicates when pivoted
    pivot_orders = pd.pivot_table(d_order, values='quantity', index=['item_name'], aggfunc=np.sum)
    #reset pivot table index to gain access to item_name column
    #reset_pivot = pivot_orders.reset_index()
    return pivot_orders
    
get_orders('salad')

,quantity
item_name,
barbacoa salad bowl,10
carnitas salad,1
carnitas salad bowl,6
chicken salad,9
chicken salad bowl,123
salad,2
steak salad,4
steak salad bowl,31
veggie salad,6
